# Script to try setting up Google Earth Engine Authentication and load some sample data

In [1]:
import ee
ee.Initialize(project='ee-ninaimmenroth')


## Load a small sample
Pick a location (e.g. Alexanderplatz) and load a small sample for a certain timeframe.

In [ ]:
roi = ee.Geometry.Point([13.4125, 52.5218]).buffer(500)

s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(roi)
      .filterDate("2023-06-01", "2023-06-30")
      .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)))

def add_ndvi(img):
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return img.addBands(ndvi)

s2_ndvi = s2.map(add_ndvi)

# This extracts random sample points from the first image

img = s2_ndvi.first()

samples = img.select("NDVI").sample(
    region=roi,
    scale=10,
    numPixels=20,   # sample size
    seed=42
)


## Download as a pandas dataframe

In [ ]:
import pandas as pd

# Convert Earth Engine FeatureCollection → client-side dict
data = samples.getInfo()

# Convert to pandas DataFrame
df = pd.DataFrame([f["properties"] for f in data["features"]])
df.head()


,NDVI
0,0.276256
1,0.091916
2,0.117857
3,0.091879
4,0.549828


In [ ]:
df.to_csv("ndvi_sample.csv", index=False)


## Download NDVI png of Berlin

In [ ]:
# geometry
berlin = ee.Geometry.Point([13.4050, 52.5200]).buffer(5000)

# select image
s2 = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(berlin)
    .filterDate("2023-06-01", "2023-09-01")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 30))
    .sort("CLOUDY_PIXEL_PERCENTAGE")
    .first()
)

# assert bands
print("bands:", s2.bandNames().getInfo())

# ndvi
ndvi = s2.normalizedDifference(["B8", "B4"]).rename("NDVI")

# style
ndvi_vis = ndvi.visualize(
    min=0.0,
    max=1.0,
    palette=['blue','white','green']
)

# thumbnail
thumb_url = ndvi_vis.getThumbURL({
    "region": berlin,
    "dimensions": 800,
    "format": "png"
})

print("Download NDVI:", thumb_url)

bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'MSK_CLASSI_OPAQUE', 'MSK_CLASSI_CIRRUS', 'MSK_CLASSI_SNOW_ICE']
Download NDVI: https://earthengine.googleapis.com/v1/projects/ee-ninaimmenroth/thumbnails/3c337696cfd68754637298942be39a4f-af80adf688908d8f562067527268478f:getPixels
